In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import itertools
from wordcloud import WordCloud, STOPWORDS

from src.features.preprocess import PreProcess

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer

import re
from gensim import corpora, models
import gensim

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /home/andy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/andy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/andy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/andy/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Read the posts csv file

In [2]:
subreddit = "computerscience"

In [3]:
df_posts = pd.read_csv("../data/raw/" + subreddit + "_posts.csv")
df_posts.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created
0,n2n0ax,New to programming or computer science? Want a...,375,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09
1,qb4bof,THIS IS NOT A TECH SUPPORT SUB OR A COMPUTER R...,404,0.97,computerscience,https://www.reddit.com/r/computerscience/comme...,54,Tech Support: /r/techsupport\n\nComputer Recom...,1.634619e+09
2,s8afrp,Ways to learn higher level math through CS?,12,0.88,computerscience,https://www.reddit.com/r/computerscience/comme...,3,"I'm a university student, and while I've taken...",1.642654e+09
3,s7yrsa,Why is there nothing between 8bit and 16bit?,23,0.82,computerscience,https://www.reddit.com/r/computerscience/comme...,19,"For example, if i want to change the image spe...",1.642622e+09
4,s7li8v,Why is Internet/transmission speed in Bit and ...,59,0.95,computerscience,https://www.reddit.com/r/computerscience/comme...,41,I am always wondering about this. Why Internet...,1.642581e+09


# Preprocess

In [4]:
PreProcess.preprocess(df_posts, 'title', lemm=True)
PreProcess.preprocess(df_posts, 'body', lemm=True)
df_posts.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created,title_word_token,title_filtered,title_stem,title_tag,title_lemm,body_word_token,body_filtered,body_stem,body_tag,body_lemm
0,n2n0ax,New to programming or computer science? Want a...,375,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]","[new, programming, computer, science]","[new, program, comput, scienc]","[[(new, JJ)], [(programming, VBG)], [(computer...","[new, program, computer, science]","[The, previous, thread, was, finally, archived...","[previous, thread, finally, archived, 500, com...","[previou, thread, final, archiv, 500, comment,...","[[(previous, JJ)], [(thread, NN)], [(finally, ...","[previous, thread, finally, archive, 500, comm..."
1,qb4bof,THIS IS NOT A TECH SUPPORT SUB OR A COMPUTER R...,404,0.97,computerscience,https://www.reddit.com/r/computerscience/comme...,54,Tech Support: /r/techsupport\n\nComputer Recom...,1.634619e+09,"[THIS, IS, NOT, A, TECH, SUPPORT, SUB, OR, A, ...","[tech, support, sub, computer, recommendation,...","[tech, support, sub, comput, recommend, sub]","[[(tech, NN)], [(support, NN)], [(sub, NN)], [...","[tech, support, sub, computer, recommendation,...","[Tech, Support, :, /r/techsupport, Computer, R...","[tech, support, rtechsupport, computer, recomm...","[tech, support, rtechsupport, comput, recommen...","[[(tech, NN)], [(support, NN)], [(rtechsupport...","[tech, support, rtechsupport, computer, recomm..."
2,s8afrp,Ways to learn higher level math through CS?,12,0.88,computerscience,https://www.reddit.com/r/computerscience/comme...,3,"I'm a university student, and while I've taken...",1.642654e+09,"[Ways, to, learn, higher, level, math, through...","[ways, learn, higher, level, math, cs]","[way, learn, higher, level, math, cs]","[[(ways, NNS)], [(learn, NN)], [(higher, JJR)]...","[way, learn, high, level, math, c]","[I, 'm, a, university, student, ,, and, while,...","[university, student, taken, math, classes, la...","[univers, student, taken, math, class, la, mul...","[[(university, NN)], [(student, NN)], [(taken,...","[university, student, take, math, class, la, m..."
3,s7yrsa,Why is there nothing between 8bit and 16bit?,23,0.82,computerscience,https://www.reddit.com/r/computerscience/comme...,19,"For example, if i want to change the image spe...",1.642622e+09,"[Why, is, there, nothing, between, 8bit, and, ...","[nothing, 8bit, 16bit]","[noth, 8bit, 16bit]","[[(nothing, NN)], [(8bit, CD)], [(16bit, CD)]]","[nothing, 8bit, 16bit]","[For, example, ,, if, i, want, to, change, the...","[example, want, change, image, specifications,...","[exampl, want, chang, imag, specif, photoshop,...","[[(example, NN)], [(want, NN)], [(change, NN)]...","[example, want, change, image, specification, ..."
4,s7li8v,Why is Internet/transmission speed in Bit and ...,59,0.95,computerscience,https://www.reddit.com/r/computerscience/comme...,41,I am always wondering about this. Why Internet...,1.642581e+09,"[Why, is, Internet/transmission, speed, in, Bi...","[internettransmission, speed, bit, storage, ca...","[internettransmiss, speed, bit, storag, capaci...","[[(internettransmission, NN)], [(speed, NN)], ...","[internettransmission, speed, bit, storage, ca...","[I, am, always, wondering, about, this, .]","[always, wondering]","[alway, wonder]","[[(always, RB)], [(wondering, VBG)]]","[always, wonder]"


# Compute Vader polarity scores for title and body of posts

In [5]:
analyzer = SentimentIntensityAnalyzer()


df_posts['title_polarity'] = df_posts['title'].apply(analyzer.polarity_scores)
df_posts['body_polarity'] = df_posts['body'].apply(analyzer.polarity_scores)
df_posts.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created,title_word_token,...,title_stem,title_tag,title_lemm,body_word_token,body_filtered,body_stem,body_tag,body_lemm,title_polarity,body_polarity
0,n2n0ax,New to programming or computer science? Want a...,375,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[new, program, comput, scienc]","[[(new, JJ)], [(programming, VBG)], [(computer...","[new, program, computer, science]","[The, previous, thread, was, finally, archived...","[previous, thread, finally, archived, 500, com...","[previou, thread, final, archiv, 500, comment,...","[[(previous, JJ)], [(thread, NN)], [(finally, ...","[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co..."
1,qb4bof,THIS IS NOT A TECH SUPPORT SUB OR A COMPUTER R...,404,0.97,computerscience,https://www.reddit.com/r/computerscience/comme...,54,Tech Support: /r/techsupport\n\nComputer Recom...,1.634619e+09,"[THIS, IS, NOT, A, TECH, SUPPORT, SUB, OR, A, ...",...,"[tech, support, sub, comput, recommend, sub]","[[(tech, NN)], [(support, NN)], [(sub, NN)], [...","[tech, support, sub, computer, recommendation,...","[Tech, Support, :, /r/techsupport, Computer, R...","[tech, support, rtechsupport, computer, recomm...","[tech, support, rtechsupport, comput, recommen...","[[(tech, NN)], [(support, NN)], [(rtechsupport...","[tech, support, rtechsupport, computer, recomm...","{'neg': 0.096, 'neu': 0.825, 'pos': 0.079, 'co...","{'neg': 0.041, 'neu': 0.885, 'pos': 0.074, 'co..."
2,s8afrp,Ways to learn higher level math through CS?,12,0.88,computerscience,https://www.reddit.com/r/computerscience/comme...,3,"I'm a university student, and while I've taken...",1.642654e+09,"[Ways, to, learn, higher, level, math, through...",...,"[way, learn, higher, level, math, cs]","[[(ways, NNS)], [(learn, NN)], [(higher, JJR)]...","[way, learn, high, level, math, c]","[I, 'm, a, university, student, ,, and, while,...","[university, student, taken, math, classes, la...","[univers, student, taken, math, class, la, mul...","[[(university, NN)], [(student, NN)], [(taken,...","[university, student, take, math, class, la, m...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.034, 'neu': 0.917, 'pos': 0.049, 'co..."
3,s7yrsa,Why is there nothing between 8bit and 16bit?,23,0.82,computerscience,https://www.reddit.com/r/computerscience/comme...,19,"For example, if i want to change the image spe...",1.642622e+09,"[Why, is, there, nothing, between, 8bit, and, ...",...,"[noth, 8bit, 16bit]","[[(nothing, NN)], [(8bit, CD)], [(16bit, CD)]]","[nothing, 8bit, 16bit]","[For, example, ,, if, i, want, to, change, the...","[example, want, change, image, specifications,...","[exampl, want, chang, imag, specif, photoshop,...","[[(example, NN)], [(want, NN)], [(change, NN)]...","[example, want, change, image, specification, ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.0, 'neu': 0.978, 'pos': 0.022, 'comp..."
4,s7li8v,Why is Internet/transmission speed in Bit and ...,59,0.95,computerscience,https://www.reddit.com/r/computerscience/comme...,41,I am always wondering about this. Why Internet...,1.642581e+09,"[Why, is, Internet/transmission, speed, in, Bi...",...,"[internettransmiss, speed, bit, storag, capaci...","[[(internettransmission, NN)], [(speed, NN)], ...","[internettransmission, speed, bit, storage, ca...","[I, am, always, wondering, about, this, .]","[always, wondering]","[alway, wonder]","[[(always, RB)], [(wondering, VBG)]]","[always, wonder]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.04, 'neu': 0.905, 'pos': 0.055, 'com..."


# Topic Modeling (LDA)

LDA Config Settings

In [6]:
# Number of topics to generate
num_topics = 5
# Number of passes the LDA should run
num_passes = 100

# Number of words to print out
num_words = 5

LDA on titles of all posts

##### LDA on title

In [7]:
posts = list(df_posts['title_lemm'])
dictionary = corpora.Dictionary(posts)
corpus = [dictionary.doc2bow(article) for article in 
          posts] # All except the last one

In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, 
                                            id2word = dictionary, 
                                            passes=num_passes)

topic_results = lda_model.print_topics(num_topics=num_topics, num_words=num_words)
topic_results

[(0,
  '0.016*"time" + 0.014*"computer" + 0.011*"machine" + 0.011*"c" + 0.009*"learn"'),
 (1,
  '0.031*"computer" + 0.021*"science" + 0.012*"algorithm" + 0.009*"hash" + 0.009*"read"'),
 (2,
  '0.015*"book" + 0.015*"computer" + 0.013*"use" + 0.011*"learn" + 0.010*"program"'),
 (3,
  '0.030*"computer" + 0.019*"question" + 0.013*"memory" + 0.012*"program" + 0.010*"know"'),
 (4,
  '0.021*"program" + 0.020*"data" + 0.014*"structure" + 0.013*"algorithm" + 0.009*"ai"')]

##### LDA on body

In [9]:
posts = list(df_posts['body_lemm'])
dictionary = corpora.Dictionary(posts)
corpus = [dictionary.doc2bow(article) for article in 
          posts] # All except the last one

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, 
                                            id2word = dictionary, 
                                            passes=num_passes)

topic_results = lda_model.print_topics(num_topics=num_topics, num_words=num_words)
topic_results

[(0,
  '0.018*"program" + 0.018*"n" + 0.013*"hello" + 0.013*"computer" + 0.009*"nt"'),
 (1,
  '0.011*"like" + 0.011*"computer" + 0.010*"work" + 0.009*"hi" + 0.009*"software"'),
 (2,
  '0.011*"want" + 0.009*"would" + 0.008*"recently" + 0.008*"get" + 0.008*"algorithm"'),
 (3,
  '0.015*"computer" + 0.013*"algorithm" + 0.010*"learn" + 0.010*"book" + 0.008*"like"'),
 (4,
  '0.014*"know" + 0.009*"case" + 0.009*"computer" + 0.009*"work" + 0.008*"anyone"')]

##### LDA on comments

In [11]:
df_comments = pd.read_csv("../data/raw/" + subreddit + "_comments.csv", lineterminator='\n')
df_comments.head()

,post_id,comment
0,n2n0ax,How late is too late to start a career in prog...
1,n2n0ax,I am a freshman at a university and haven't be...
2,n2n0ax,I'm still in highschool but really interested ...
3,n2n0ax,"This is probably a common question, but how we..."
4,n2n0ax,I am planning on starting a CS major this fall...


In [12]:
PreProcess.preprocess(df_comments, 'comment', lemm=True)
df_comments.head()

,post_id,comment,comment_word_token,comment_filtered,comment_stem,comment_tag,comment_lemm
0,n2n0ax,How late is too late to start a career in prog...,"[How, late, is, too, late, to, start, a, caree...","[late, late, start, career, programming]","[late, late, start, career, program]","[[(late, RB)], [(late, RB)], [(start, NN)], [(...","[late, late, start, career, program]"
1,n2n0ax,I am a freshman at a university and haven't be...,"[I, am, a, freshman, at, a, university, and, h...","[freshman, university, nt, able, work, side, p...","[freshman, univers, nt, abl, work, side, proje...","[[(freshman, NN)], [(university, NN)], [(nt, N...","[freshman, university, nt, able, work, side, p..."
2,n2n0ax,I'm still in highschool but really interested ...,"[I, 'm, still, in, highschool, but, really, in...","[still, highschool, really, interested, comput...","[still, highschool, realli, interest, comput, ...","[[(still, RB)], [(highschool, NN)], [(really, ...","[still, highschool, really, interested, comput..."
3,n2n0ax,"This is probably a common question, but how we...","[This, is, probably, a, common, question, ,, b...","[probably, common, question, well, coding, boo...","[probabl, common, question, well, code, bootca...","[[(probably, RB)], [(common, JJ)], [(question,...","[probably, common, question, well, cod, bootca..."
4,n2n0ax,I am planning on starting a CS major this fall...,"[I, am, planning, on, starting, a, CS, major, ...","[planning, starting, cs, major, fall]","[plan, start, cs, major, fall]","[[(planning, NN)], [(starting, VBG)], [(cs, NN...","[planning, start, c, major, fall]"


##### Compute polarity of comments

In [13]:
df_comments['comment_polarity'] = df_comments['comment'].apply(analyzer.polarity_scores)
df_comments.head()

,post_id,comment,comment_word_token,comment_filtered,comment_stem,comment_tag,comment_lemm,comment_polarity
0,n2n0ax,How late is too late to start a career in prog...,"[How, late, is, too, late, to, start, a, caree...","[late, late, start, career, programming]","[late, late, start, career, program]","[[(late, RB)], [(late, RB)], [(start, NN)], [(...","[late, late, start, career, program]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,n2n0ax,I am a freshman at a university and haven't be...,"[I, am, a, freshman, at, a, university, and, h...","[freshman, university, nt, able, work, side, p...","[freshman, univers, nt, abl, work, side, proje...","[[(freshman, NN)], [(university, NN)], [(nt, N...","[freshman, university, nt, able, work, side, p...","{'neg': 0.087, 'neu': 0.837, 'pos': 0.076, 'co..."
2,n2n0ax,I'm still in highschool but really interested ...,"[I, 'm, still, in, highschool, but, really, in...","[still, highschool, really, interested, comput...","[still, highschool, realli, interest, comput, ...","[[(still, RB)], [(highschool, NN)], [(really, ...","[still, highschool, really, interested, comput...","{'neg': 0.0, 'neu': 0.823, 'pos': 0.177, 'comp..."
3,n2n0ax,"This is probably a common question, but how we...","[This, is, probably, a, common, question, ,, b...","[probably, common, question, well, coding, boo...","[probabl, common, question, well, code, bootca...","[[(probably, RB)], [(common, JJ)], [(question,...","[probably, common, question, well, cod, bootca...","{'neg': 0.02, 'neu': 0.847, 'pos': 0.134, 'com..."
4,n2n0ax,I am planning on starting a CS major this fall...,"[I, am, planning, on, starting, a, CS, major, ...","[planning, starting, cs, major, fall]","[plan, start, cs, major, fall]","[[(planning, NN)], [(starting, VBG)], [(cs, NN...","[planning, start, c, major, fall]","{'neg': 0.0, 'neu': 0.968, 'pos': 0.032, 'comp..."


##### Merge posts and comments together

In [14]:
df_merge = df_posts.merge(df_comments, how='left', left_on='post_id', right_on='post_id')
df_merge.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created,title_word_token,...,body_lemm,title_polarity,body_polarity,comment,comment_word_token,comment_filtered,comment_stem,comment_tag,comment_lemm,comment_polarity
0,n2n0ax,New to programming or computer science? Want a...,375,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co...",How late is too late to start a career in prog...,"[How, late, is, too, late, to, start, a, caree...","[late, late, start, career, programming]","[late, late, start, career, program]","[[(late, RB)], [(late, RB)], [(start, NN)], [(...","[late, late, start, career, program]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,n2n0ax,New to programming or computer science? Want a...,375,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co...",I am a freshman at a university and haven't be...,"[I, am, a, freshman, at, a, university, and, h...","[freshman, university, nt, able, work, side, p...","[freshman, univers, nt, abl, work, side, proje...","[[(freshman, NN)], [(university, NN)], [(nt, N...","[freshman, university, nt, able, work, side, p...","{'neg': 0.087, 'neu': 0.837, 'pos': 0.076, 'co..."
2,n2n0ax,New to programming or computer science? Want a...,375,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co...",I'm still in highschool but really interested ...,"[I, 'm, still, in, highschool, but, really, in...","[still, highschool, really, interested, comput...","[still, highschool, realli, interest, comput, ...","[[(still, RB)], [(highschool, NN)], [(really, ...","[still, highschool, really, interested, comput...","{'neg': 0.0, 'neu': 0.823, 'pos': 0.177, 'comp..."
3,n2n0ax,New to programming or computer science? Want a...,375,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co...","This is probably a common question, but how we...","[This, is, probably, a, common, question, ,, b...","[probably, common, question, well, coding, boo...","[probabl, common, question, well, code, bootca...","[[(probably, RB)], [(common, JJ)], [(question,...","[probably, common, question, well, cod, bootca...","{'neg': 0.02, 'neu': 0.847, 'pos': 0.134, 'com..."
4,n2n0ax,New to programming or computer science? Want a...,375,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1037,The previous thread was finally archived with ...,1.619890e+09,"[New, to, programming, or, computer, science, ?]",...,"[previous, thread, finally, archive, 500, comm...","{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...","{'neg': 0.018, 'neu': 0.815, 'pos': 0.167, 'co...",I am planning on starting a CS major this fall...,"[I, am, planning, on, starting, a, CS, major, ...","[planning, starting, cs, major, fall]","[plan, start, cs, major, fall]","[[(planning, NN)], [(starting, VBG)], [(cs, NN...","[planning, start, c, major, fall]","{'neg': 0.0, 'neu': 0.968, '

In [15]:
# Check the number of posts with no comments
print("There are {} posts with no comments".format(df_merge.comment.isna().sum()))
print("There are {} posts with no content after filtering".format(len(df_merge[df_merge['comment_stem'].str.len() == 0])))

There are 45 posts with no comments
There are 30 posts with no content after filtering


##### LDA for all comments

In [16]:
# Drop the posts that do not have any comments
df_merge_dropped = df_merge[df_merge['comment_lemm'].str.len() > 0]

posts = list(df_merge_dropped['comment_lemm'])
dictionary = corpora.Dictionary(posts)
corpus = [dictionary.doc2bow(article) for article in 
          posts] # All except the last one

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, 
                                            id2word = dictionary, 
                                            passes=num_passes)

topic_results = lda_model.print_topics(num_topics=num_topics, num_words=num_words)
topic_results

[(0, '0.012*"c" + 0.012*"nt" + 0.012*"use" + 0.011*"really" + 0.011*"start"'),
 (1,
  '0.014*"one" + 0.010*"algorithm" + 0.009*"like" + 0.009*"delete" + 0.008*"would"'),
 (2,
  '0.045*"computer" + 0.026*"science" + 0.014*"code" + 0.012*"software" + 0.009*"go"'),
 (3, '0.016*"n" + 0.014*"1" + 0.010*"get" + 0.010*"yes" + 0.008*"number"'),
 (4,
  '0.015*"program" + 0.014*"learn" + 0.012*"like" + 0.011*"would" + 0.011*"use"')]